## Variables and parameters are local
Assignment statements within a function create <strong>local variables</strong> for that function on the right hand side of the assignment. These variables only exist inside the function and can't be used outside the function scope.<br>
Consider the below example where we print the value of y which is local only to the function ``square``. This variable ``y`` is a local variable because the assignment statement ``y = x * x`` is present in the function definition.

In [1]:
def square(x):
    y = x * x
    return y

z = square(10)
print(y)

NameError: name 'y' is not defined

The variable ``y`` only exists while the function is being executed — we call this its <strong>lifetime</strong>. When the execution of the function terminates (returns), the local variables are destroyed.<br>
Formal parameters are also local and act like local variables. For example, the lifetime of ``x`` begins when square is called, and its lifetime ends when the function completes its execution.<br>

So it is not possible for a function to set some local variable to a value, complete its execution, and then when it is called again next time, recover the local variable. Each call of the function creates new local variables, and their lifetimes expire when the function returns to the caller.<br>
```python 
    v1 += 1
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
NameError: name 'v1' is not defined
    def foo():
        v1 += 1
    foo()
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
    File "<stdin>", line 2, in foo
UnboundLocalError: local variable 'v1' referenced before assignment
```
<br>
In the code above, notice and understand the different error messages. The local variables are created at the same time the local namespace is created (as seen in the function foo). That is <strong>any</strong> variable that is assigned to anywhere in the function gets added to the local namespace immediately but it will remain <strong>unbound</strong> until the assignment statement is executed.



## Global Variables:
Variables which are at the top-level, not inside any function definition, are called **global variables**.<br>
It is legal for a function to access a global variable. However, this is considered **bad form** by nearly all programmers and should be avoided.

In [2]:
def badsquare(x):
    y = x ** power
    return y

power = 2
result = badsquare(10)
print("y =", result)

y = 100


Although the above code is a case of bad implementation, the point to learn here is how does Python search for a variable:
- First, Python looks at the variables that are defined as **local variables** in the function. We call this the **local scope**. 
- If the variable name is not found in the local scope, then Python looks at the **global variables**, or **global scope**.

As shown in the above example, since ``power`` is not defined within the function definition (local scope), Python searches for ``power`` in the global scope and finds that the value of ``power`` is 2. As mentioned previously, this way of implementation is bad. The appropriate way to write this function would be to pass ``power`` as a parameter.

In [5]:
# Assignment statements in the local function cannot change variables defined outside the function.
def powerof(x, p):
    power = p
    y = x ** power
    return y

power = 3
result = powerof(10, 2)
print("result:", result)
print("power:", power)    # Should still be 3 as changing local var y shouldn't change the global var y

result: 100
power: 3


In [7]:
# To really change the value of a global variable inside a function, we explicitly declare the variable to be global
# However, this is not recommended and not a good practice as well because this changes the global variable itself
def powerof(x, p):
    global power
    power = p
    y = x ** power
    return y

power = 3
result = powerof(10, 2)
print("result:", result)
print("power:", power)   # Should now be changed to 2 as we explicitly declared global in the func def causing a value change

result: 100
power: 2


Functional decomposition is a technique where we can make use of functions and call functions withing them. An example is shown below where the first function called ``square`` simply computes the square of a given number. The second function called ``sum_of_squares`` makes use of square to compute the sum of three numbers that have been squared.

In [2]:
def square(x):
    y = x * x
    return y

def sum_of_squares(x,y,z):
    a = square(x)
    b = square(y)
    c = square(z)
    return a + b + c

a = -5
b = 2
c = 10
print("Sum of three squares with sides {}, {}, {} is {}".format(a, b, c, sum_of_squares(a, b, c)))
    

Sum of three squares with sides -5, 2, 10 is 129


Even though this is a pretty simple idea, in practice this example illustrates many very important Python concepts, including **local** and **global** variables along with **parameter passing**. Note that the body of square is not executed until it is called from inside the sum_of_squares function for the first time.<br>
Also notice that whenever ``square`` is called, there are two groups of local variables, one for ``square`` and one for ``sum_of_squares``. Each group of local variables is called a **stack frame**.

## Passing Mutable objects:
An assignment to a **formal parameter** inside a function never affects the argument in the caller.
However, if you are passing a **mutable object**, such as a list, to a function, and the function alters the object’s state, that state change will be visible to the caller when the function returns.

In [11]:
def double(y):
    # Since we are changing the formal parameter, this won't affect the arguments at all
    num = 2 * y

def changeit(lst):
    # But here, since we are changing the items inside of the list, mylst and lst point to the same object due to which we
    # could see mylst getting changed
    lst[0] = "Michigan"
    lst[1] = "Wolverines"

num = 5
double(num)
print("num:", num)
    
mylst = ['our', 'students', 'are', 'awesome']
changeit(mylst)
print("mylst:", mylst)

num: 5
mylst: ['Michigan', 'Wolverines', 'are', 'awesome']


However, there is a difference between assigning to a slot of a list, and assigning to the list variable itself, the former changes the list passed as an argument `mylst` whereas the latter doesn't. 

Basically any kind of list operations that affect the items in a list would change the list passed as an argument, whereas any kind of list reassignment (changing list as a whole) wouldn't affect the argumentative list at all.

In [7]:
def changeit(lst):
    # We try changing the list as a whole, In this case, we could see mylst would not change at all.
    # This happens because after assigning lst to a new list, the object reference gets changed.
    lst = ["Michigan", "Wolverines"]
    
mylst = ['our', 'students', 'are', 'awesome']
changeit(mylst)
print("mylst:", mylst)

mylst: ['our', 'students', 'are', 'awesome']


We say that the function ``changeit`` has a **side effect** on the list object that is passed to it when the actual list is changed. **Global variables** are another way to have side effects.

For example, we could make the function ``double`` have a side effect by having the variable ``num`` global.

In [10]:
def double(y):
    global num
    num = 2 * y

num = 5
double(num)
print("num:", num)

num: 10


In general, any lasting effect that occurs in a function, not through its return value, is called a **side effect**. There are three ways to have side effects:

- **Printing** out a value. This doesn’t change any objects or variable bindings, but it does have a potential lasting effect outside the function execution, because a person might see the output and be influenced by it.
- Changing the value of a **mutable object**.
- Changing the binding of a **global variable**.

It is best to avoid side effects. The way to avoid using side effects is to **use return values instead**.

In [18]:
def double(y):
    return 2 * y

num = 5
# The value of num does not change until we assign the return value of double(num) to num again.
double(num)
print("num:", num)

num: 5


You can use the same coding pattern to avoid confusing side effects with sharing of mutable objects. 

To do that, 
- explicitly make a copy of an object and pass the copy in to the function. 
- Then return the modified copy and reassign it to the original variable if you want to save the changes. 

The built-in ``list`` function, which takes a sequence as a parameter and returns a new list, works to copy an existing list. For dictionaries, you can similarly call the ``dict`` function, passing in a dictionary to get a copy of the dictionary back as a return value.

In [15]:
def changeit(lst):
    lst[0] = "Michigan"
    lst[1] = "Wolverines"
    return lst

mylst = ['106', 'students', 'are', 'awesome']

# Create a copy of mylst using the list function. This would point to a diff obj ref
newlst = changeit(list(mylst))

print(mylst)
print(newlst)

['106', 'students', 'are', 'awesome']
['Michigan', 'Wolverines', 'are', 'awesome']


# Advanced Functions:
## Optional Parameters:
Sometimes it is convenient to have **optional parameters** that can be specified or omitted. 
- When an optional parameter is omitted from a function invocation, the formal parameter is bound to a **default value**.
- When an optional parameter is included from a function invocation, the formal parameter is bound to the **given value**.

In [3]:
initial = 7
def f(x, y = 3, z = initial):
    return "x, y, z are: {}, {}, {}".format(x, y, z)

print(f(2))        # y and z would be set to the default value provided (3 and 7(initial))
print(f(3, 5))     # z would be set to the default value provided (7(initial))
print(f(6, 7, 8))  # None of the default values would be set, the given values would be the values of x, y and z

x, y, z are: 2, 3, 7
x, y, z are: 3, 5, 7
x, y, z are: 6, 7, 8


- The default values are determined at the time of function definition and not invocation. In the example above, if we set the value of initial before making any of the function calls, the value of ``initial`` would still be 7.
- If the default value is set to a mutable object, such as a list or a dictionary, that object will be shared in all invocations of the function because mutable objects point to the same object. This can get very confusing, so never set a default value that is a mutable object.

## Keyword Parameters:
Keyword parameters are just another way of invoking functions with parameters defined as a keyword instead of passing them positionally as we saw in the code above.<br>
This is particularly convenient when there are several optional parameters and you want to provide a value for one of the later parameters while not providing a value for the earlier ones.<br>
With keyword arguments, some of the values can be of the form ``paramname = <expr>`` instead of just ``<expr>``.<br>
**Note:**
- When you have ``paramname = <expr>`` in a function **definition**, it is defining the **default value** for a parameter when no value is provided in the invocation; 
- When you have ``paramname = <expr>`` in the **invocation**, it is supplying a value, **overriding the default** for that paramname.
- Positional parameters (one which are not keyworded) should always be present **before** keyword parameters in the function invocation.

In [4]:
initial = 7
def f(x, y = 3, z = initial):
    return "x, y, z are: {}, {}, {}".format(x, y, z)

print(f(2))        # y and z would be set to the default value provided (3 and 7(initial))
print(f(3, z = 5))     # y would be set to the default value provided (3)
print(f(6, z = 9, y = 8))  # None of the default values would be set, the given values would be the values of x, y and z

x, y, z are: 2, 3, 7
x, y, z are: 3, 3, 5
x, y, z are: 6, 8, 9


Keyword arguments can also be used with the .format() method. This is particularly useful when we need to insert the same value into a string multiple times. Although, the usual way of passing positonal arguments also works!

In [6]:
names = ["Jack","Jill","Mary"]
for n in names:
    print("'{0}!' she yelled. '{0}! {0}, {1}!'".format(n,"say hello"))

'Jack!' she yelled. 'Jack! Jack, say hello!'
'Jill!' she yelled. 'Jill! Jill, say hello!'
'Mary!' she yelled. 'Mary! Mary, say hello!'


## Lambda expressions:
The syntax of a lambda expression is the word “lambda” followed by parameter names, separated by commas but not inside (parentheses), followed by a colon and then an expression. ``lambda arguments: expression`` yields a function object.
This unnamed object behaves just like the function object.
```python
def func(args):
    return ret_val

# This is equivalent to:
func = lambda args: ret_val
```

In the typical function, we have to use the keyword ``return`` to send back the value. In a lambda function, that is not necessary - whatever is placed after the colon is what will be returned.

In [12]:
def f(x):
    return x - 1

# f is bound to a function object
print("f:", f)
print("type(f):", type(f))
print("f(3) --", f(3))

# lambda func also processes a function object and since it is an anonymous function, we see <function <lambda>...
print("\nlambda x: x-2 --", lambda x: x-2)
print("type(lambda x: x-2) --", type(lambda x: x-2))
print("(lambda x: x-2)(6) --", (lambda x: x-2)(6))

# We can alternatively assign the lambda func to a value
lf = lambda x: x-2
print("\nlf(6) --", lf(6))

f: <function f at 0x000001A0634E0D30>
type(f): <class 'function'>
f(3) -- 2

lambda x: x-2 -- <function <lambda> at 0x000001A0634E0EE0>
type(lambda x: x-2) -- <class 'function'>
(lambda x: x-2)(6) -- 4

lf(6) -- 4
